In [719]:

import numpy as np
import scipy.integrate
import scipy.optimize
from bokeh.plotting import figure, output_file, show

import bokeh.plotting
import bokeh.io
from bokeh.models import Span


# We will use Matplotlib to make a 3D plot
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import panel as pn
pn.extension()

bokeh.io.output_notebook()

Loading BokehJS ...

In [945]:
def Self_regenerate_model(molecules,t,alpha,beta,K,KTL,KTX,lamb1,lamb2,Kdr_0,Kdr_1,Kdr_2,Kdr_3):
    R,dT,dG,mT,mG,pT,pG,TX,TL = molecules
    return np.array(
        [
            -alpha*TX*dT/(dT+dG+KTX)*pT*R/(R+K)-alpha*TX*dG/(dT+dG+KTX)*pT*R/(R+K)-beta*TL*mT/(mT+mG+KTL)*R/(R+K)-beta*TL*mG/(mT+mG+KTL)*R/(R+K)+(100-R)*Kdr_1,
            -Kdr_2*dT,
            -Kdr_2*dG,
            alpha*TX*dT/(dT+dG+KTX)*pT*R/(R+K)-Kdr_3*mT,
            alpha*TX*dG/(dT+dG+KTX)*pT*R/(R+K)-Kdr_3*mG,
            beta*TL*(mT/(mT+mG+KTL))*(R/(R+K))-Kdr_3*pT*Kdr_4-(10)*Kdr_0,
            beta*TL*mG/(mT+mG+KTL)*R/(R+K)-Kdr_3*pG,
            -lamb1*TX,
            -lamb1*TL
        ]
    )
    

In [946]:
#common parameters 
alpha=0.7;
beta=0.07;
K=1.0;
KTL=10;
KTX=1;
lamb1=0;
lamb2=0;

#initial condition
R0=100.0;
dT0=0.5;
dG0=2;
mT0=0.0;
mG0=0.0;
pT0=1;
pG0=0.0;
TX0=1;
TL0=1;

molecules0 = [R0,dT0,dG0,mT0,mG0,pT0,pG0,TX0,TL0]


#stage 1 kick start
t = np.linspace(0,240,17)

Kdr_0=0.012
Kdr_1=0.012
Kdr_2=0
Kdr_3=0.012
Kdr_4=0

result = scipy.integrate.odeint(Self_regenerate_model, molecules0, t, args=(alpha,beta,K,KTL,KTX,lamb1,lamb2,Kdr_0,Kdr_1,Kdr_2,Kdr_3))
#update parameters 
molecules0 = result.transpose()[:,-1]
result_all = result

#stage 2 regenerate
t = np.linspace(240, 960, 49)

Kdr_0=0
Kdr_1=0.012
Kdr_2=0
Kdr_3=0.012
Kdr_4=1

result = scipy.integrate.odeint(Self_regenerate_model, molecules0, t, args=(alpha,beta,K,KTL,KTX,lamb1,lamb2,Kdr_0,Kdr_1,Kdr_2,Kdr_3))
molecules0 = result.transpose()[:,-1]

result_all = np.append(result_all,result[1:],axis=0)


#stage 3 washout
t = np.linspace(960, 1800, 57)

Kdr_0=0
Kdr_1=0.012
Kdr_2=0.012
Kdr_3=0.012
Kdr_4=1

result = scipy.integrate.odeint(Self_regenerate_model, molecules0, t, args=(alpha,beta,K,KTL,KTX,lamb1,lamb2,Kdr_0,Kdr_1,Kdr_2,Kdr_3))

result_all = np.append(result_all,result[1:],axis=0)
R,dT,dG,mT,mG,pT,pG,TX,TL = result_all.transpose()




In [947]:
t = np.linspace(0, 30, 121)
p = bokeh.plotting.figure(
    plot_width=800,
    plot_height=200,
    x_axis_label="t",
    y_axis_label=" concentration",
    y_axis_type="linear",

)

colors = bokeh.palettes.d3["Category10"][5]

# Populate glyphs

p.line(
    t, pT, line_width=2, color=colors[0], legend_label="pG"
)

vline1 = Span(location=4, dimension='height', line_color='black', line_width=1,line_dash='dashed')
vline2 = Span(location=16, dimension='height', line_color='black', line_width=1,line_dash='dashed')
p.add_layout(vline1)
p.add_layout(vline2)

p.legend.click_policy="hide"

show(p)


In [899]:
np.shape(result_all)

(121, 9)

In [494]:
R,dT,dG,mT,mG,pT,pG,TX,TL = result_all.transpose()

In [499]:
t = np.linspace(0, 24, 96)
p = bokeh.plotting.figure(
    plot_width=800,
    plot_height=400,
    x_axis_label="t",
    y_axis_label="dimensionless concentration",
    y_axis_type="linear",

)

colors = bokeh.palettes.d3["Category10"][3]

# Populate glyphs
p.line(
    t, pT, line_width=2, color=colors[0]
)
show(p)


In [595]:
[1,2,3,4,5,5,6][1:]

[2, 3, 4, 5, 5, 6]